In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

path='/content/drive/MyDrive/DIP/Assignment05/Images'

model = VGG16(weights='imagenet', include_top=False)
#model.summary()

58900480/58889256 [==============================] - 0s 0us/step


In [2]:
subdir=os.listdir(path)
train_subdir=[x for x in subdir if 'train' in x]
test_subdir=[x.replace('train','test') for x in train_subdir]

In [ ]:

#airplanes = class 0 (train data=30 , test data = 20)
#bikes = class 1 (train data=30 , test data = 20)
#faces = class 2  (train data=30 , test data = 20)
#cars = class 3 (train data=30 , test data = 20)

#train and test data stacked in this order in array

**Feature Extraction for Train Data**

In [3]:
vgg16_feature_list_train = []
y_train=[]
#loading train images from folders and find features using model
for idx, dirname in tqdm(enumerate(train_subdir)):
    filenames=os.listdir(path+'/'+dirname)
    for i, fname in enumerate(filenames):
        img_path=path+'/'+dirname+'/'+fname
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list_train.append(vgg16_feature_np.flatten())
        y_train.append(idx)

4it [01:35, 23.94s/it]


**Feature Extraction for Test Data**

In [4]:
vgg16_feature_list_test = []
y_test=[]
#loading test images from folders and find features using model
for idx, dirname in tqdm(enumerate(test_subdir)):
    filenames=os.listdir(path+'/'+dirname)
    for i, fname in enumerate(filenames):
        img_path=path+'/'+dirname+'/'+fname
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list_test.append(vgg16_feature_np.flatten())
        y_test.append(idx)

4it [01:01, 15.27s/it]


In [5]:
#Convert to numpy array
vgg16_feature_list_np_train = np.array(vgg16_feature_list_train)
vgg16_feature_list_np_test = np.array(vgg16_feature_list_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

**KNN**

In [15]:
# Locate the K-most nearest neighbors
def Knn(train, test_row, num_neighbors):
  distances = list()
  for index,train_row in enumerate(train):
    dist = np.sqrt(np.sum(np.square(np.subtract(test_row,train_row))))
    distances.append((index, dist))
  distances.sort(key=lambda tup: tup[1])
  neighbors = []
  for i in range(num_neighbors):
    neighbors.append(distances[i][0])
  return neighbors

# Make a class prediction with KNN
def predict_classification(train, test_row, num_neighbors,y_train):
  neighbors = Knn(train, test_row, num_neighbors)
  output_values = [y_train[index] for index in neighbors]
  prediction = max(set(output_values), key=output_values.count)
  return prediction

#predict for all test data
y_pred=np.copy(y_test)
for i in range(len(y_test)):
  y_pred[i]=predict_classification(vgg16_feature_list_np_train,vgg16_feature_list_np_test[i].T,4,y_train)

In [39]:
print(' Accuracy Of Model : ',np.sum((y_pred-y_test)==0)/len(y_test))
print(' Accuracy in Class 0 (Airplane) : ',np.sum((y_pred[:20]-y_test[:20])==0)/20)
print(' Accuracy in Class 1 (bikes) : ',np.sum((y_pred[20:40]-y_test[20:40])==0)/20)
print(' Accuracy in Class 2 (faces) : ',np.sum((y_pred[40:60]-y_test[40:60])==0)/20)
print(' Accuracy in Class 3 (cars) : ',np.sum((y_pred[60:80]-y_test[60:80])==0)/20)

 Accuracy Of Model :  0.9875
 Accuracy Of Class 0 (Airplane) :  1.0
 Accuracy Of Class 1 (bikes) :  0.95
 Accuracy Of Class 2 (faces) :  1.0
 Accuracy Of Class 3 (cars) :  1.0
